In [68]:
import pandas as pd
file_path = '/kaggle/input/dl-original/training/sentences.tsv'
df = pd.read_csv(file_path, sep='\t')
import pandas as pd
file_path = '/kaggle/input/dl-original/training/labels.tsv'
df1 = pd.read_csv(file_path, sep='\t')



In [69]:
def adjust_features(row):
    for i in range(len(row) - 1):
        if row[i] == 0.5 and row[i + 1] == 0.5:
            row[i] = 0
            row[i + 1] = 1
    return row

df1 = df1.apply(adjust_features, axis=1)

/tmp/ipykernel_34/3750566648.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[i] == 0.5 and row[i + 1] == 0.5:
/tmp/ipykernel_34/3750566648.py:4: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[i] = 0
/tmp/ipykernel_34/3750566648.py:5: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[i + 1] = 1


In [70]:
import pandas as pd
common_columns = ['Text-ID', 'Sentence-ID']
merged_df = pd.merge(df, df1, on=common_columns, how='inner')
merged_df

,Text-ID,Sentence-ID,Text,Self-direction: thought attained,Self-direction: thought constrained,Self-direction: action attained,Self-direction: action constrained,Stimulation attained,Stimulation constrained,Hedonism attained,...,Benevolence: caring attained,Benevolence: caring constrained,Benevolence: dependability attained,Benevolence: dependability constrained,Universalism: concern attained,Universalism: concern constrained,Universalism: nature attained,Universalism: nature constrained,Universalism: tolerance attained,Universalism: tolerance constrained
0,BG_002,1,"ДЪРЖАВАТА СРЕЩУ ЛОЗАН ПАНОВ ЗА СЪМНЕНИЯ, ЧЕ Е ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BG_002,2,КВ.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BG_002,3,М МОРСКО БУНГАЛО НА ЗАНИЖЕНИ ЦЕНИ Лавината от ...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BG_002,4,Вече месеци наред той е обект на проверка за к...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BG_002,5,Там от години прокуратурата се явява съдружник...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40662,TR_M_022,12,Kolay kazanç sağlayan ve istihdam öngörmeyen r...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40663,TR_M_022,13,"Vergi tabana yayılacak, yoksul kesimin vergi y...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40664,TR_M_022,14,Sosyal refahı artıracak destekleyici düzenleme...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40665,TR_M_022,15,Yoksulluktan en fazla etkilenen eğitim düzeyi ...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
df = merged_df

In [72]:
import pandas as pd
df = df.drop(['Text-ID', 'Sentence-ID'], axis=1)


In [73]:
df = df.dropna()
df['Text'].replace('', pd.NA, inplace=True)
df = df.dropna()

/tmp/ipykernel_34/1000494590.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Text'].replace('', pd.NA, inplace=True)


In [74]:
import pandas as pd
X = df['Text']
Y = df.drop(['Text'], axis=1)

In [75]:
X = X.values
Y = Y.values

In [76]:
X = X.tolist()
Y = Y.tolist()

In [77]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

class BertMultiLabelClassifier(nn.Module):
    def __init__(self, num_labels):
        super(BertMultiLabelClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(768, num_labels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs['pooler_output']
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        probabilities = self.sigmoid(logits)
        return probabilities


In [78]:
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
from transformers import AdamW, get_linear_schedule_with_warmup, BertTokenizer
from transformers import BertForSequenceClassification  # Assuming a classification task
from tqdm import tqdm
import torch.nn as nn
import torch


tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenized_X = [tokenizer(sentence, padding=True, truncation=True, max_length=512, return_tensors='pt') for sentence in X]
padded_input_ids = pad_sequence([item['input_ids'][0] for item in tokenized_X], batch_first=True, padding_value=tokenizer.pad_token_id)
padded_attention_mask = pad_sequence([item['attention_mask'][0] for item in tokenized_X], batch_first=True, padding_value=0)  # Assuming pad_token_id is 0
dataset = TensorDataset(padded_input_ids, padded_attention_mask, torch.tensor(Y))
train_loader = DataLoader(dataset, batch_size=8, shuffle=True,pin_memory=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = BertMultiLabelClassifier(num_labels=38)
model.to(device)

criterion = nn.BCELoss()
optimizer = AdamW(model.parameters(), lr=2e-5)

num_epochs = 3
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    
    with tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as t:
        for batch in t:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to('cuda')
            attention_mask = attention_mask.to('cuda')
            labels = labels.to('cuda')
            
            optimizer.zero_grad()
            output = model(input_ids, attention_mask)
            loss = criterion(output, labels.float())
            total_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            # Update progress bar
            t.set_postfix({'loss': total_loss / (t.n + 1)})

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}')

    


cuda


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/3: 100%|██████████| 5084/5084 [32:31<00:00,  2.60batch/s, loss=0.0757]


Epoch 1/3, Average Loss: 0.0757


Epoch 2/3: 100%|██████████| 5084/5084 [32:33<00:00,  2.60batch/s, loss=0.0603]


Epoch 2/3, Average Loss: 0.0603


Epoch 3/3: 100%|██████████| 5084/5084 [32:32<00:00,  2.60batch/s, loss=0.0519]

Epoch 3/3, Average Loss: 0.0519


In [79]:
checkpoint = {'model': model,
              'state_dict': model.state_dict(),
              'optimizer' : optimizer.state_dict()}

torch.save(checkpoint, 'checkpoint.pth')

In [80]:
import pandas as pd
file_path = '/kaggle/input/dl-original/validation/sentences.tsv'
df = pd.read_csv(file_path, sep='\t')
import pandas as pd
file_path = '/kaggle/input/dl-original/validation/labels.tsv'
df1 = pd.read_csv(file_path, sep='\t')

def adjust_features(row):
    for i in range(len(row) - 1):
        if row[i] == 0.5 and row[i + 1] == 0.5:
            row[i] = 0
            row[i + 1] = 1
    return row

df1 = df1.apply(adjust_features, axis=1)

import pandas as pd
common_columns = ['Text-ID', 'Sentence-ID']
merged_df = pd.merge(df, df1, on=common_columns, how='inner')
merged_df

df = merged_df
import pandas as pd
df = df.drop(['Text-ID', 'Sentence-ID'], axis=1)

df = df.dropna()
df['Text'].replace('', pd.NA, inplace=True)
df = df.dropna()

import pandas as pd
X = df['Text']
Y = df.drop(['Text'], axis=1)

X = X.values
Y = Y.values

X = X.tolist()
Y = Y.tolist()

/tmp/ipykernel_34/3364832504.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[i] == 0.5 and row[i + 1] == 0.5:
/tmp/ipykernel_34/3364832504.py:11: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[i] = 0
/tmp/ipykernel_34/3364832504.py:12: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[i + 1] = 1
/tmp/ipykernel_34/3364832504.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series 

In [81]:
pip install --upgrade transformers


Note: you may need to restart the kernel to use updated packages.


In [82]:
X_eval = X
Y_eval = Y

In [83]:
from sklearn.metrics import classification_report
import torch

# Load the saved model
checkpoint = torch.load('checkpoint.pth')
loaded_model = checkpoint['model']
loaded_model.load_state_dict(checkpoint['state_dict'])
loaded_model.to(device)
loaded_model.eval()

# Preprocess the new dataset
# Assuming you have a new dataset X_eval, Y_eval
tokenized_X_eval = [tokenizer(sentence, padding=True, truncation=True, max_length=512, return_tensors='pt') for sentence in X_eval]
padded_input_ids_eval = pad_sequence([item['input_ids'][0] for item in tokenized_X_eval], batch_first=True, padding_value=tokenizer.pad_token_id)
padded_attention_mask_eval = pad_sequence([item['attention_mask'][0] for item in tokenized_X_eval], batch_first=True, padding_value=0)
eval_dataset = TensorDataset(padded_input_ids_eval, padded_attention_mask_eval, torch.tensor(Y_eval))
eval_loader = DataLoader(eval_dataset, batch_size=8, shuffle=False, pin_memory=True)

# Evaluate the model
all_preds = []
all_labels = []

with torch.no_grad():
    for batch_eval in eval_loader:
        input_ids_eval, attention_mask_eval, labels_eval = batch_eval
        input_ids_eval = input_ids_eval.to(device)
        attention_mask_eval = attention_mask_eval.to(device)
        labels_eval = labels_eval.to(device)

        output_eval = loaded_model(input_ids_eval, attention_mask_eval)
        preds_eval = (output_eval > 0.5).float()  # Assuming threshold of 0.5 for binary classification
        all_preds.append(preds_eval.cpu().numpy())
        all_labels.append(labels_eval.cpu().numpy())

all_preds = np.concatenate(all_preds, axis=0)
all_labels = np.concatenate(all_labels, axis=0)




In [84]:
label_names = [
    "Self-direction: thought attained",
    "Self-direction: thought constrained",
    "Self-direction: action attained",
    "Self-direction: action constrained",
    "Stimulation attained",
    "Stimulation constrained",
    "Hedonism attained",
    "Hedonism constrained",
    "Benevolence: caring attained",
    "Benevolence: caring constrained",
    "Benevolence: dependability attained",
    "Benevolence: dependability constrained",
    "Universalism: concern attained",
    "Universalism: concern constrained",
    "Universalism: nature attained",
    "Universalism: nature constrained",
    "Universalism: tolerance attained",
    "Universalism: tolerance constrained"
]




In [85]:
import numpy as np

all_preds_unique = np.unique(all_labels)
print(all_preds_unique)


[0. 1.]


In [86]:
# Generate and print classification report
classification_rep = classification_report(all_labels, all_preds)
print(classification_rep)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       118
           1       0.00      0.00      0.00        34
           2       0.00      0.00      0.00       308
           3       0.00      0.00      0.00       100
           4       0.00      0.00      0.00       305
           5       0.00      0.00      0.00        52
           6       0.00      0.00      0.00        71
           7       0.00      0.00      0.00        28
           8       0.55      0.16      0.25       623
           9       0.00      0.00      0.00       245
          10       0.24      0.03      0.06       368
          11       0.00      0.00      0.00       170
          12       0.38      0.06      0.11       412
          13       0.33      0.07      0.11       246
          14       0.00      0.00      0.00       102
          15       0.00      0.00      0.00       150
          16       0.00      0.00      0.00        65
          17       0.64    

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
